In [1]:
import pandas as pd
#pd.set_option('max_rows', 10)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as statsmodels # useful stats package with regression functions
import seaborn as sns #plotting package
import zipfile

In [2]:
# Importing the data from https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators
# ONLY RUN ONCE


# from urllib.request import urlopen
# from io import BytesIO
# from zipfile import ZipFile

#pathname = "/Users/feliphlvo/Documents/Minerva/2022 Fall/Tutorial/DS4SG-Global-Inequality"
pathname = "/Users/norika_machome/GitHub/DS4SG-Global-Inequality"

# def download_and_unzip(url, extract_to='.'):
#     print("Downloading")
#     http_response = urlopen(url)
#     zipfile = ZipFile(BytesIO(http_response.read()))
#     print("Unziping")
#     zipfile.extractall(path=extract_to)
#     print("Done")


# download_and_unzip("https://databank.worldbank.org/data/download/WDI_csv.zip",
#                    pathname+"/Data/Raw")


In [3]:
#from urllib.request import urlopen
#from io import BytesIO
#from zipfile import ZipFile
#
#pathname2 = "/Users/norika_machome/GitHub/DS4SG-Global-Inequality"
#
#def download_and_unzip(url, extract_to='.'):
#     print("Downloading")
#     http_response = urlopen(url)
#     zipfile = ZipFile(BytesIO(http_response.read()))
#     print("Unziping")
#     zipfile.extractall(path=extract_to)
#     print("Done")
#
#download_and_unzip("https://databank.worldbank.org/data/download/WDI_csv.zip",pathname2+"/Data/Raw")

In [4]:
wdi_data = pd.read_csv(pathname + "/Data/Raw/WDIData.csv", delimiter=",")
#wdi_codebook = pd.read_csv(pathname+"/Data/Raw/WDISeries.csv", delimiter=",")

In [5]:
def create_df_indicator(indicator_code, write_to = None):
    # Select only indicators from topics
    topic_df = wdi_data.loc[wdi_data["Indicator Code"].isin(indicator_code)]
    # Melt dataset (making all date columns one variable)
    long_indicator_df = topic_df.melt(id_vars=["Country Name",'Country Code','Indicator Code',"Indicator Name"], 
        var_name="Date")
    long_indicator_df = long_indicator_df[long_indicator_df["value"].notna()]
    long_indicator_df = long_indicator_df[long_indicator_df["Date"]!= "Unnamed: 66"]
    long_indicator_df["Date"] = pd.to_numeric(long_indicator_df["Date"])
    
    # Write to csv
    if write_to:
        long_indicator_df.to_csv(pathname + "/Data/Final/" + write_to + ".csv", index = False)
        print("Written to:", pathname + "/Data/Final/" + write_to + ".csv")
    return long_indicator_df

test_df = create_df_indicator(["SL.UEM.NEET.ZS","SE.SEC.TCAQ.ZS", "SP.DYN.LE00.IN","SP.DYN.LE00.FE.IN","SP.DYN.LE00.MA.IN", "SP.REG.BRTH.ZS","SI.DST.FRST.10",'SI.DST.10TH.10',"NY.GDP.PCAP.CD","AG.LND.FRST.K2","EG.ELC.COAL.ZS"],"indicators")
test_df = test_df[test_df['value'].notna()]
test_df

Written to: /Users/norika_machome/GitHub/DS4SG-Global-Inequality/Data/Final/indicators.csv


,Country Name,Country Code,Indicator Code,Indicator Name,Date,value
3,Africa Eastern and Southern,AFE,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,162.726325
6,Africa Eastern and Southern,AFE,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",1960,44.300571
7,Africa Eastern and Southern,AFE,SP.DYN.LE00.MA.IN,"Life expectancy at birth, male (years)",1960,41.177958
8,Africa Eastern and Southern,AFE,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",1960,42.716053
14,Africa Western and Central,AFW,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,107.930722
...,...,...,...,...,...,...
181371,West Bank and Gaza,PSE,NY.GDP.PCAP.CD,GDP per capita (current US$),2021,3663.969055
181377,West Bank and Gaza,PSE,SL.UEM.NEET.ZS,"Share of youth not in education, employment or...",2021,31.510000
181382,"Yemen, Rep.",YEM,NY.GDP.PCAP.CD,GDP per capita (current US$),2021,690.759273
181393,Zambia,ZMB,NY.GDP.PCAP.CD,GDP per capita (current US$),2021,1120.630171


In [8]:
test_df["Indicator Name"].unique()


array(['GDP per capita (current US$)',
       'Life expectancy at birth, female (years)',
       'Life expectancy at birth, male (years)',
       'Life expectancy at birth, total (years)',
       'Electricity production from coal sources (% of total)',
       'Income share held by highest 10%',
       'Income share held by lowest 10%',
       'Share of youth not in education, employment or training, total (% of youth population)',
       'Forest area (sq. km)',
       'Trained teachers in secondary education (% of total teachers)',
       'Completeness of birth registration (%)'], dtype=object)

In [6]:
wdi_data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.936004,17.337896,17.687093,18.140971,18.491344,18.825520,19.272212,19.628009,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.499471,6.680066,6.859110,7.016238,7.180364,7.322294,7.517191,7.651598,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.855399,38.046781,38.326255,38.468426,38.670044,38.722783,38.927016,39.042839,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.794160,32.001027,33.871910,38.880173,40.261358,43.061877,44.270860,45.803485,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,18.663502,17.633986,16.464681,24.531436,25.345111,27.449908,29.641760,30.404935,NaN,NaN


In [7]:
merged = pd.merge(wdi_data, wdi_codebook, how='inner',
                  left_on='Indicator Name', right_on='Indicator Name')
merged['Topic']

NameError: name 'wdi_codebook' is not defined

In [ ]:
#Checking which indicators don't have a defined Topic to see if it's okay to omit them for our study
#df1 = merged[merged['Topic'].isnull()]
#display(set(df1["Indicator Name"]))

In [ ]:
#showing all the available columns 
pd.set_option('max_columns', None)
print(merged.columns.tolist())

OptionError: 'Pattern matched multiple keys'

Creating Separate datasets for each type of indicator (facilitates understanding and reduces size for upload to Github)

In [ ]:
def create_df_topic(topics, write_to = None):
    # Select only indicators from topics
    topic_df = merged.loc[merged["Topic"].isin(topics)]
    # Melt dataset (making all date columns one variable)
    long_topic_df = topic_df.melt(id_vars=["Country Name", "Indicator Name",'Country Code', 'Topic', 'Indicator Code'], 
        var_name="Date")
    # Write to csv
    if write_to:
        long_topic_df.to_csv(pathname2 + "/Data/Final/" + write_to + ".csv", index = False)
        print("Written to:", pathname2 + "/Data/Final/" + write_to + ".csv")
    return long_topic_df


In [ ]:
#education_df = create_df_topic(['Education: Outcomes','Education: Participation'], "Education")
#poverty_df = create_df_topic(['Poverty: Income distribution','Poverty: Poverty rates'], "Poverty")
#mortality_df = create_df_topic(['Health: Mortality'], "Mortality")
#labor_df = create_df_topic(["Social Protection & Labor: Unemployment", "Social Protection & Labor: Labor force structure"], "Labor")
#environmnet_df = create_df_topic(["Environment: Land use", "Environment: Emissions"], "Environment")

In [ ]:
#checking out all of the indicators
#print(set(education_df["Indicator Name"]))

{'School enrollment, primary, male (% gross)', 'School enrollment, primary, female (% net)', 'Adjusted net enrollment rate, primary (% of primary school age children)', 'Adjusted net enrollment rate, primary, female (% of primary school age children)', 'Primary completion rate, female (% of relevant age group)', 'School enrollment, primary, female (% gross)', 'Children out of school, primary, male', 'Children out of school, male (% of male primary school age)', 'Secondary education, general pupils (% female)', 'Secondary education, pupils', 'School enrollment, secondary, female (% net)', 'School enrollment, primary (% net)', 'School enrollment, secondary (% gross)', 'Literacy rate, adult total (% of people ages 15 and above)', 'Children out of school (% of primary school age)', 'Primary education, duration (years)', 'Educational attainment, at least completed primary, population 25+ years, male (%) (cumulative)', 'Educational attainment, at least completed upper secondary, population 2

In [ ]:
education_df.describe()

,Country Name,Indicator Name,Country Code,Topic,Indicator Code,Date,value
count,1832208,1832208,1832208,1832208,1832208,1832208,607883
unique,266,84,266,2,84,82,344405
top,Africa Eastern and Southern,"Adjusted net enrollment rate, primary (% of pr...",AFE,Education: Participation,SE.PRM.TENR,1960,CC BY-4.0
freq,6888,21812,6888,959728,21812,22344,22344


In [ ]:
#checking to see if any of the rows have non-NA values
#selected_rows = long_df[~long_df['value'].isnull()]
#print(selected_rows)
len(education_df)

1832208

In [ ]:
education_df


,Country Name,Indicator Name,Country Code,Topic,Indicator Code,Date,value
0,Africa Eastern and Southern,"Adjusted net enrollment rate, primary (% of pr...",AFE,Education: Participation,SE.PRM.TENR,1960,NaN
1,Africa Western and Central,"Adjusted net enrollment rate, primary (% of pr...",AFW,Education: Participation,SE.PRM.TENR,1960,NaN
2,Arab World,"Adjusted net enrollment rate, primary (% of pr...",ARB,Education: Participation,SE.PRM.TENR,1960,NaN
3,Caribbean small states,"Adjusted net enrollment rate, primary (% of pr...",CSS,Education: Participation,SE.PRM.TENR,1960,NaN
4,Central Europe and the Baltics,"Adjusted net enrollment rate, primary (% of pr...",CEB,Education: Participation,SE.PRM.TENR,1960,NaN
...,...,...,...,...,...,...,...
1832203,Virgin Islands (U.S.),"Secondary education, vocational pupils (% female)",VIR,Education: Participation,SE.SEC.ENRL.VO.FE.ZS,Unnamed: 20,NaN
1832204,West Bank and Gaza,"Secondary education, vocational pupils (% female)",PSE,Education: Participation,SE.SEC.ENRL.VO.FE.ZS,Unnamed: 20,NaN
1832205,"Yemen, Rep.","Secondary education, vocational pupils (% female)",YEM,Education: Participation,SE.SEC.ENRL.VO.FE.ZS,Unnamed: 20,NaN
1832206,Zambia,"Secondary education, vocational pupils (% female)",ZMB,Education: Participation,SE.SEC.ENRL.VO.FE.ZS,Unnamed: 20,NaN
